In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import pandas as pd
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split

## Cargar Datos

In [ ]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder, StandardScaler

df = pd.read_csv("/content/drive/MyDrive/datos_limpios/cultivos_enriquecidos_espanol2.csv")

if len(df) != 1700:
    raise ValueError(f"El dataset debe tener exactamente 1,700 filas. Actualmente tiene {len(df)} filas.")

train_size = int(len(df) * 0.85)
test_size = len(df) - train_size

# Seleccionar datos de forma aleatoria para entrenamiento
df_train = df.sample(n=train_size, random_state=42).reset_index(drop=True)

# Usar el resto para prueba
df_test = df.drop(df_train.index).reset_index(drop=True)

# Validar que ambos conjuntos tienen datos
if len(df_train) == 0 or len(df_test) == 0:
    raise ValueError("Los conjuntos de entrenamiento o prueba están vacíos. Revisa el tamaño del dataset.")

# Separar características y etiquetas
X_train = df_train.drop(columns=["label"]).values
y_train = df_train["label"].values

X_test = df_test.drop(columns=["label"]).values
y_test = df_test["label"].values

# Codificar las etiquetas
label_encoder = LabelEncoder()
y_train = label_encoder.fit_transform(y_train)
y_test = label_encoder.transform(y_test)

# Normalizar las características
scaler = StandardScaler()

# Aplicar el escalador
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

print("Datos de entrenamiento:", X_train.shape)
print("Datos de prueba:", X_test.shape)

Datos de entrenamiento: (1445, 7)
Datos de prueba: (255, 7)


## Crear un Dataset Personalizado

In [ ]:
class CropDataset(Dataset):
    def __init__(self, features, labels):
        self.features = torch.tensor(features, dtype=torch.float32)
        self.labels = torch.tensor(labels, dtype=torch.long)

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        return self.features[idx], self.labels[idx]

# Crear datasets de entrenamiento y prueba
train_dataset = CropDataset(X_train, y_train)
test_dataset = CropDataset(X_test, y_test)

# Crear dataloaders
batch_size = 32
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

## Definir el Modelo

In [ ]:
class SimplifiedCropModel(nn.Module):
    def __init__(self, num_classes):
        super(SimplifiedCropModel, self).__init__()
        self.fc1 = nn.Linear(7, 64)
        self.fc2 = nn.Linear(64, num_classes)

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)
        return x

model = SimplifiedCropModel(num_classes=len(label_encoder.classes_))

## Entrenamiento del Modelo

In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Entrenar
num_epochs = 300
for epoch in range(num_epochs):
    model.train()
    epoch_loss = 0

    for inputs, labels in train_loader:
        outputs = model(inputs)
        loss = criterion(outputs, labels)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        epoch_loss += loss.item()

    # Imprimir pérdida promedio por época
    if (epoch + 1) % 10 == 0:
        print(f'Epoch [{epoch + 1}/{num_epochs}], Loss: {epoch_loss / len(train_loader):.4f}')

Epoch [10/300], Loss: 0.8009
Epoch [20/300], Loss: 0.7066
Epoch [30/300], Loss: 0.6599
Epoch [40/300], Loss: 0.6527
Epoch [50/300], Loss: 0.6390
Epoch [60/300], Loss: 0.6267
Epoch [70/300], Loss: 0.6137
Epoch [80/300], Loss: 0.6185
Epoch [90/300], Loss: 0.6056
Epoch [100/300], Loss: 0.6027
Epoch [110/300], Loss: 0.5917
Epoch [120/300], Loss: 0.5891
Epoch [130/300], Loss: 0.5919
Epoch [140/300], Loss: 0.5807
Epoch [150/300], Loss: 0.5720
Epoch [160/300], Loss: 0.5674
Epoch [170/300], Loss: 0.5640
Epoch [180/300], Loss: 0.5682
Epoch [190/300], Loss: 0.5598
Epoch [200/300], Loss: 0.5642
Epoch [210/300], Loss: 0.5531
Epoch [220/300], Loss: 0.5540
Epoch [230/300], Loss: 0.5517
Epoch [240/300], Loss: 0.5423
Epoch [250/300], Loss: 0.5372
Epoch [260/300], Loss: 0.5291
Epoch [270/300], Loss: 0.5322
Epoch [280/300], Loss: 0.5261
Epoch [290/300], Loss: 0.5251
Epoch [300/300], Loss: 0.5244


## Evaluación del Modelo

In [ ]:
model.eval()
correct = 0
total = 0

with torch.no_grad():
    for inputs, labels in test_loader:
        outputs = model(inputs)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

accuracy = 100 * correct / total
print(f'Accuracy on the test set: {accuracy:.2f}%')

Accuracy on the test set: 54.12%


## Realizar una predicción de prueba

In [ ]:
sample = [[28,37,28,32.13409675,50.52559148,6.097869767,98.63333684]]
sample = scaler.transform(sample)
sample = torch.tensor(sample, dtype=torch.float32)

model.eval()
output = model(sample)
_, predicted_label = torch.max(output, 1)
print("Predicted Label:", label_encoder.inverse_transform(predicted_label.numpy()))

Predicted Label: ['Mango']


In [ ]:
torch.save(model.state_dict(), "crop_classifier.pth")